In [133]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from itertools import permutations

import numpy as np

In [134]:
PROBLEM_SIZE = 5
NUM_SETS = 10 #number of tiles
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

print(SETS)

(array([ True,  True, False,  True, False]), array([False,  True,  True,  True, False]), array([ True, False,  True, False, False]), array([ True, False, False, False, False]), array([ True, False,  True, False, False]), array([False, False, False, False, False]), array([False, False, False,  True,  True]), array([False, False, False, False, False]), array([False, False, False,  True,  True]), array([False,  True,  True, False,  True]))


In [135]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


def check_num(min_tiles, state):
    for i in permutations(state.not_taken, r = min_tiles):
        new_state = State(state.taken, state.not_taken)
        for j in i:
            new_state = State(new_state.taken ^ {j}, new_state.not_taken ^ {j})

        if goal_check(new_state):
            return True

    return False

In [136]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [137]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action}, current_state.not_taken ^ {action}
        )

        #COST (g) = number of tiles taken starting from the root.
        #HEURISTIC (h) = min number of tiles I need to cover all the sets. This number is <= than the distance.

        min_tiles = 0

        while min_tiles != distance(new_state): 
            min_tiles += 1

            if check_num(min_tiles, new_state):
                break


        frontier.put((len(new_state.taken) + min_tiles, new_state))
        print(f"g={len(new_state.taken)} , distance = {distance(new_state)},  h={min_tiles} , sum={len(new_state.taken) + min_tiles}, {new_state}")
    _, current_state = frontier.get()
    print(f"STEP {counter}:  {current_state}")

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

g=1 , distance = 2,  h=1 , sum=2, State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9})
g=1 , distance = 2,  h=2 , sum=3, State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9})
g=1 , distance = 3,  h=2 , sum=3, State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9})
g=1 , distance = 4,  h=2 , sum=3, State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9})
g=1 , distance = 3,  h=2 , sum=3, State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9})
g=1 , distance = 5,  h=2 , sum=3, State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9})
g=1 , distance = 3,  h=2 , sum=3, State(taken={6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9})
g=1 , distance = 5,  h=2 , sum=3, State(taken={7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9})
g=1 , distance = 3,  h=2 , sum=3, State(taken={8}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9})
g=1 , distance = 2,  h=1 , sum=2, State(taken={9}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8})
STEP 1:  State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9})
g=2 , distance = 1,  h=1 , sum=3, S

In [138]:
current_state

State(taken={0, 9}, not_taken={1, 2, 3, 4, 5, 6, 7, 8})

In [139]:
goal_check(current_state)

True